In [2]:
# !aws codeartifact login --tool pip --domain cirrus-ml-ds-domain --domain-owner 813736554012 --repository cirrus-ml-ds-shared-repo
# !pip install awswrangler --quiet
# !pip install lightgbm --quiet
# !pip install category_encoders --quiet
# !pip install imbalanced-learn --quiet


In [10]:
import boto3
import awswrangler as wr
import pandas as pd
from lightgbm import LGBMClassifier
import numpy as np
from utils_determine_feature_type import determine_feature_data_types
from utils_split import make_splits, split_data, make_subsplit
from utiles_preprocessing import preprocessing_pipeline_step

boto3.setup_default_session(region_name="eu-north-1")



 ****** PandasSimpleImputer > fit ******** 


In [4]:
df = wr.athena.read_sql_query('SELECT * FROM "customerone_mock_data_rl"."master";', 
                              database="customerone_mock_data_rl",
                              workgroup="dev-athena-workgroup",
                             )
#                               workgroup="dev-ds-athena-workgroup",
df

,customer_id,current_dt,tgt_xsell_cust_voice_to_fixed,dmgrphc_b_age_bucket_txt,dmgrphc_b_zip_code_cd,dmgrphc_b_gender_typ,dmgrphc_b_city_txt,dmgrphc_b_country_txt,dmgrphc_b_no_of_dependents_val,dmgrphc_b_habits_desc_txt,...,cust_prd_hld_d_vas_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_mobilevoicesubscription_closed_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_eom_total_bill_amt_m0_val,rev_m_eom_total_bill_amt_m1_val,rev_m_eom_total_bill_amt_m2_val,partition_0
0,100127,2021-09-24,<NA>,57,40,M,South Martinton,Colombia,4,Good,...,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
1,100136,2021-06-18,<NA>,33,36,U,Webermouth,Benin,2,Bad,...,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
2,100152,2021-07-12,<NA>,20,84,U,South Lindsayhaven,Eritrea,1,Bad,...,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,xsell_cust_voice_to_fixed
3,100154,2021-05-25,<NA>,55,73,U,Kyleview,Comoros,3,Bad,...,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,xsell_cust_voice_to_fixed
4,100158,2021-08-26,<NA>,47,54,M,Aaronburgh,Denmark,1,Bad,...,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40204,100947,2021-05-16,<NA>,47,35,M,New Kathryn,Japan,1,Good,...,NaN,NaN,1.000000,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
40205,10095,2021-09-03,<NA>,64,17,M,Port Tammy,Saint Pierre and Miquelon,3,Bad,...,NaN,NaN,1.000000,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
40206,10097,2021-08-26,<NA>,60,19,M,West Connieport,Australia,2,Bad,...,1.000000,NaN,1.000000,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
40207,100971,2021-10-22,<NA>,62,38,F,Meganborough,India,4,Bad,...,1.000000,1.0,NaN,NaN,0,0,0.0,0.0,0.0,xsell_cust_voice_to_fixed


In [5]:
target_col_name = 'tgt_xsell_cust_voice_to_fixed'


# Train

In [7]:
# skipped sample_training_df()
# skipped drop_invalid_features()
# skipped filter_features()

# skipped to_pandas() and just filtered the null target rows:
df_training = df[df['tgt_xsell_cust_voice_to_fixed'].notnull()]
df_training_idx = df_training.reset_index(drop=True)
print(f"{df_training.shape}, {df_training_idx.shape}")

(5896, 74), (5896, 74)


In [8]:
df_training_idx

,customer_id,current_dt,tgt_xsell_cust_voice_to_fixed,dmgrphc_b_age_bucket_txt,dmgrphc_b_zip_code_cd,dmgrphc_b_gender_typ,dmgrphc_b_city_txt,dmgrphc_b_country_txt,dmgrphc_b_no_of_dependents_val,dmgrphc_b_habits_desc_txt,...,cust_prd_hld_d_vas_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_mobilevoicesubscription_closed_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_eom_total_bill_amt_m0_val,rev_m_eom_total_bill_amt_m1_val,rev_m_eom_total_bill_amt_m2_val,partition_0
0,100279,2021-09-07,0,25,33,F,Kingberg,Aruba,2,Bad,...,1.878788,NaN,NaN,NaN,0,0,151.136185,0.0,0.0,xsell_cust_voice_to_fixed
1,100279,2021-09-07,0,25,33,F,Kingberg,Aruba,2,Bad,...,1.878788,NaN,NaN,NaN,0,0,151.136185,0.0,0.0,xsell_cust_voice_to_fixed
2,100279,2021-09-07,0,25,33,F,Kingberg,Aruba,2,Bad,...,1.878788,NaN,NaN,NaN,0,0,151.136185,0.0,0.0,xsell_cust_voice_to_fixed
3,100397,2021-08-27,0,43,88,M,Jessicafurt,Syrian Arab Republic,2,Bad,...,NaN,NaN,NaN,1.0,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
4,100397,2021-08-27,0,43,88,M,Jessicafurt,Syrian Arab Republic,2,Bad,...,NaN,NaN,NaN,1.0,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5891,100280,2021-06-02,0,52,64,F,North Charles,Niue,2,Bad,...,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
5892,100463,2021-05-29,0,32,23,U,South Kellyland,Liberia,4,Bad,...,NaN,NaN,1.0,NaN,0,0,268.582890,0.0,0.0,xsell_cust_voice_to_fixed
5893,10052,2021-09-09,0,49,82,F,Kaylafort,Jordan,4,Bad,...,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
5894,100620,2021-07-21,0,26,70,F,West Samantha,Qatar,0,Bad,...,NaN,NaN,NaN,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed


# features

In [11]:
spine_params_determine_feature_data_types = {'keys': ['customer_id'],
                                             'date_column': 'current_dt',
                                             'product_holdings_filter': {'product_category': 'fixedbroadband'},
                                             'is_deepsell': 'N'}

target_params_determine_feature_data_types = {'target_variable_column': 'tgt_xsell_cust_voice_to_fixed'
    , 'lead_time_window': '1d'
    , 'target_window': '45d'
    , 'product_activation_filter': {'product_sub_category': 'voice'}
    , 'campaign_keys': ['customer_id']
    , 'campaign_filter': {'campaign_name': ["C-452-O-06 Korsförsäljning Telia Life 2.0 - Sälja mobilt"
        , "b2c_cross-sell_pp_crossSellPpToBb"
        , "b2c_cross-sell_pp_crossSellPpToBb_oldContent"
        , "b2c_cross-sell_pp_crossSellPpToBb_REM1"
        , "C-700-O-03 Cross-sell Mobile to Broadband customers (TM only)"
        , "C-700-O-01 Cross-sell Mobile to Broadband customers (A)"
        , "C-752-O GEOF 2021 - X-sell PP"
        , "C-652-O Black Friday Erbjudande 2019  Mobilt till BB-kund - activity 1"
        , "C-700-O-02 Cross-sell Mobile to Broadband customers (B)"
        , "b2c_cross-sell_Pp_PpToBb_default"
        , "C-752-O GEOF 2021 - Xsell PP"
        , "b2c_cross-sell_Pp_PpToBb_simOnly"
        , "b2c_cross-sell_Pp_PpToBb_samS215G"
        , "b2c_cross-sell_Pp_PpToBb_iphone12Mini"
        , "b2c_cross-sell_Pp_PpToBb_iphoneSE"
        , "b2c_cross-sell_Pp_PpToBb_iphone12"
        , "b2c_cross-sell_Pp_PpToBb_samS20FE5G"
        , "b2c_cross-sell_Pp_PpToBb_sonyXp10lll"
        , "b2c_cross-sell_Pp_PpToBb_default_short_8pm"
        , "b2c_cross-sell_Pp_PpToBb_default_8pm"]
        , 'customer_actioned_flg_column': {'Email': 'actioned_ind'}}}

feature_dict = determine_feature_data_types(df_training,
                                            spine_params_determine_feature_data_types,
                                            target_params_determine_feature_data_types)
print(f"len(feature_dict)={len(feature_dict)}, \nlen(feature_dict['numeric'])={len(feature_dict['numeric'])}, \nlen(feature_dict['categorical'])={len(feature_dict['categorical'])}")

features = feature_dict['categorical'] + feature_dict['numeric']

len(feature_dict)=3, 
len(feature_dict['numeric'])=62, 
len(feature_dict['categorical'])=3


In [15]:
# Splitting data
split_train_test = make_splits(df_training_idx, target_col_name)

print(f"\n\n{df_training.shape}, {df_training_idx.shape}, {split_train_test.shape}\n")
print(f"split_train_test['split'].value_counts(dropna=False):\n {split_train_test['split'].value_counts(dropna=False)}\n")
print(f"df_training_idx['split'].value_counts(dropna=False):\n {df_training_idx['split'].value_counts(dropna=False)}\n")
print(f"split_train_test['iteration_id'].value_counts(dropna=False):\n {split_train_test['iteration_id'].value_counts(dropna=False)}")

split_train_test



(5896, 74), (5896, 76), (5896, 76)

split_train_test['split'].value_counts(dropna=False):
 TRAIN    4654
TEST     1242
Name: split, dtype: int64

df_training_idx['split'].value_counts(dropna=False):
 NaN    5896
Name: split, dtype: int64

split_train_test['iteration_id'].value_counts(dropna=False):
 0    5896
Name: iteration_id, dtype: int64


,customer_id,current_dt,tgt_xsell_cust_voice_to_fixed,dmgrphc_b_age_bucket_txt,dmgrphc_b_zip_code_cd,dmgrphc_b_gender_typ,dmgrphc_b_city_txt,dmgrphc_b_country_txt,dmgrphc_b_no_of_dependents_val,dmgrphc_b_habits_desc_txt,...,cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_eom_total_bill_amt_m0_val,rev_m_eom_total_bill_amt_m1_val,rev_m_eom_total_bill_amt_m2_val,partition_0,iteration_id,split
0,100279,2021-09-07,0,25,33,F,Kingberg,Aruba,2,Bad,...,NaN,NaN,0,0,151.136185,0.0,0.0,xsell_cust_voice_to_fixed,0,TEST
1,100279,2021-09-07,0,25,33,F,Kingberg,Aruba,2,Bad,...,NaN,NaN,0,0,151.136185,0.0,0.0,xsell_cust_voice_to_fixed,0,TEST
2,100279,2021-09-07,0,25,33,F,Kingberg,Aruba,2,Bad,...,NaN,NaN,0,0,151.136185,0.0,0.0,xsell_cust_voice_to_fixed,0,TEST
3,100397,2021-08-27,0,43,88,M,Jessicafurt,Syrian Arab Republic,2,Bad,...,NaN,1.0,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TEST
4,100397,2021-08-27,0,43,88,M,Jessicafurt,Syrian Arab Republic,2,Bad,...,NaN,1.0,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5891,100280,2021-06-02,0,52,64,F,North Charles,Niue,2,Bad,...,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN
5892,100463,2021-05-29,0,32,23,U,South Kellyland,Liberia,4,Bad,...,1.0,NaN,0,0,268.582890,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
5893,10052,2021-09-09,0,49,82,F,Kaylafort,Jordan,4,Bad,...,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TEST
5894,100620,2021-07-21,0,26,70,F,West Samantha,Qatar,0,Bad,...,NaN,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN


In [16]:
splitter_args = {'test_size': 0.1, 'random_state': 42}
subsplit_params = {"iteration_col_name": 'iteration_id', "split_col_name": "split", 'source_split': 'TRAIN',
                   'target_splits': ['TRAIN', 'CAL']}

split_train_test_cal = make_subsplit(split_train_test, subsplit_params, splitter_args, target_col_name)
print(f"\n\nsplit_train_test_cal['split'].value_counts(dropna=False):\n {split_train_test_cal['split'].value_counts(dropna=False)}")
split_train_test_cal



_extract_group_column_arg: group_col_name=None
_extract_group_column_arg: target_col_name=tgt_xsell_cust_voice_to_fixed
split_train_test_cal['split'].value_counts(dropna=False):
 TRAIN    4188
TEST     1242
CAL       466
Name: split, dtype: int64


,customer_id,current_dt,tgt_xsell_cust_voice_to_fixed,dmgrphc_b_age_bucket_txt,dmgrphc_b_zip_code_cd,dmgrphc_b_gender_typ,dmgrphc_b_city_txt,dmgrphc_b_country_txt,dmgrphc_b_no_of_dependents_val,dmgrphc_b_habits_desc_txt,...,cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_eom_total_bill_amt_m0_val,rev_m_eom_total_bill_amt_m1_val,rev_m_eom_total_bill_amt_m2_val,partition_0,iteration_id,split
0,100279,2021-09-07,0,25,33,F,Kingberg,Aruba,2,Bad,...,NaN,NaN,0,0,151.136185,0.0,0.0,xsell_cust_voice_to_fixed,0,TEST
1,100279,2021-09-07,0,25,33,F,Kingberg,Aruba,2,Bad,...,NaN,NaN,0,0,151.136185,0.0,0.0,xsell_cust_voice_to_fixed,0,TEST
2,100279,2021-09-07,0,25,33,F,Kingberg,Aruba,2,Bad,...,NaN,NaN,0,0,151.136185,0.0,0.0,xsell_cust_voice_to_fixed,0,TEST
3,100397,2021-08-27,0,43,88,M,Jessicafurt,Syrian Arab Republic,2,Bad,...,NaN,1.0,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TEST
4,100397,2021-08-27,0,43,88,M,Jessicafurt,Syrian Arab Republic,2,Bad,...,NaN,1.0,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5891,100280,2021-06-02,0,52,64,F,North Charles,Niue,2,Bad,...,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN
5892,100463,2021-05-29,0,32,23,U,South Kellyland,Liberia,4,Bad,...,1.0,NaN,0,0,268.582890,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
5893,10052,2021-09-09,0,49,82,F,Kaylafort,Jordan,4,Bad,...,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TEST
5894,100620,2021-07-21,0,26,70,F,West Samantha,Qatar,0,Bad,...,NaN,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
